In [280]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [281]:
df = sns.load_dataset("mpg")

X_train, y_train = df[~df["horsepower"].isna()][["displacement", "acceleration"]], df[~df["horsepower"].isna()]["horsepower"]
X_pred = df[df["horsepower"].isna()][["displacement", "acceleration"]]

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_pred)
y_pred = np.round(y_pred)
df.loc[X_pred.index, "horsepower"] = y_pred
df = pd.get_dummies(df.drop("name", axis = 1), columns = ["origin"])
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin_europe,origin_japan,origin_usa
0,18.0,8,307.0,130.0,3504,12.0,70,0,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,0,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,0,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,0,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,0,0,1


In [282]:
X, y = df.drop(["mpg"], axis = 1).values, df["mpg"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

In [283]:
scaler = StandardScaler()
#scaler = MinMaxScaler()

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_val = scaler.transform(X_val)
scaled_X_test = scaler.transform(X_test)

In [300]:
class VectorRegressor:
    def __init__(self, n = 100, hidden_layers = False, activation = "sigmoid", random_state = None, verbose = 0):

        self.n = n // 2 * 2
        self.nets = []
        self.best_net = -1
        self.best_result = None
        self.validation_loss_history = []
        self.training_loss_history = []
        self.mutation_sigma = 0

        if activation == "sigmoid":
            self.activation_function = lambda x: 1 / (1 + np.exp(-x))
        elif activation == "relu":
            self.activation_function = lambda x: np.maximum(0, x)
        elif activation == "leaky_relu":
            self.activation_function = lambda x: np.maximum(0.1 * x, x)
        
        if hidden_layers:
            self.layers = hidden_layers + [1]
        else:
            self.layers = [1]
        
        if random_state != None:
            np.random.seed(random_state)

        self.verbose = verbose

    
    def fit(self, X_train, y_train, epochs = 100, validation_data = False, verbose = 0):
        X_train = np.c_[np.ones(X_train.shape[0]), X_train]

        if validation_data:
            X_val, y_val = validation_data

        self.layers = [X_train.shape[1]] + self.layers

        self.y_preds = np.zeros((self.n, y_train.shape[0]))
        self.nets_loss = np.zeros(self.n)
        self.sorted_indecies = np.zeros(self.n)

        self.weights = []

        for i in range(len(self.layers) - 1):
            self.weights += [np.random.uniform(-3, 3, (self.n, self.layers[i], self.layers[i + 1]))]

        for epoch in range(epochs):
            forward_pass = X_train.T
            
            for j in range(0, len(self.layers) - 2):
                forward_pass = self.activation_function(self.weights[j].transpose(0, 2, 1) @ forward_pass)

            forward_pass = self.weights[-1].transpose(0, 2, 1) @ forward_pass
            self.y_preds = forward_pass.reshape(self.n, -1)

            self.nets_loss = np.mean(np.abs(self.y_preds - y_train), axis = 1)

            self.sorted_indecies = np.argsort(self.nets_loss)

            self.mutation_sigma = 0.1 + 5 * 1 / math.exp(epoch / ((epochs + 1) / (10 * math.log10(epochs + 1))))

            for j in range(0, len(self.layers) - 1):
                self.weights[j][self.sorted_indecies[50::2]] = np.mean((self.weights[j][self.sorted_indecies[:50:2]], self.weights[j][self.sorted_indecies[1:51:2]]), axis = 0) + np.random.normal(0, self.mutation_sigma, (self.n // 4, self.layers[j], self.layers[j + 1]))
                self.weights[j][self.sorted_indecies[51::2]] = np.mean((self.weights[j][self.sorted_indecies[:50:2]], self.weights[j][self.sorted_indecies[1:51:2]]), axis = 0) + np.random.normal(0, self.mutation_sigma, (self.n // 4, self.layers[j], self.layers[j + 1]))

            if self.best_net != self.sorted_indecies[0]:
                self.best_net = self.sorted_indecies[0]
                self.training_loss_history += [self.nets_loss[self.best_net]]
                
                if validation_data:
                    self.validation_loss_history += [np.mean(np.abs(y_val - self.predict(X_val)))]
                    if verbose == 1:
                        print(f"Epoch {epoch} - loss: {self.training_loss_history[-1]} - val_loss: {self.validation_loss_history[-1]}")
                else:
                    if verbose == 1:
                        pass
                        print(f"Epoch {epoch} - loss: {self.training_loss_history[-1]}")


    def predict(self, X):
        X = np.c_[np.ones(X.shape[0]), X]

        forward_pass = X.T
        for j in range(0, len(self.layers) - 2):
            forward_pass = self.activation_function(self.weights[j][self.best_net].T @ forward_pass)

        forward_pass = self.weights[-1][self.best_net].T @ forward_pass
        return forward_pass.reshape(-1)


In [301]:
vregressor = VectorRegressor(n = 100, hidden_layers = [6], activation = "relu", random_state = 42)
vregressor.fit(scaled_X_train, y_train, epochs = 1000, validation_data = (scaled_X_val, y_val), verbose = 1)

Epoch 0 - loss: 9.102721013798714 - val_loss: 8.398943305675033
Epoch 25 - loss: 8.57143305946211 - val_loss: 8.351760926959304
Epoch 27 - loss: 7.008406893645711 - val_loss: 7.078457510766609
Epoch 41 - loss: 6.863639484033477 - val_loss: 7.502665028701311
Epoch 47 - loss: 6.180237851149276 - val_loss: 4.577349783085332
Epoch 54 - loss: 5.947949521366185 - val_loss: 5.71314795147191
Epoch 58 - loss: 5.302623581963404 - val_loss: 5.199747199713962
Epoch 61 - loss: 5.007625696947729 - val_loss: 5.197120808074201
Epoch 62 - loss: 4.956147982732969 - val_loss: 4.387979008222061
Epoch 66 - loss: 4.952389188188683 - val_loss: 6.060077579986136
Epoch 69 - loss: 4.564585517164943 - val_loss: 3.9724024869876673
Epoch 70 - loss: 4.508589764471477 - val_loss: 3.540674122308976
Epoch 74 - loss: 3.3642325457566606 - val_loss: 3.1813853100013896
Epoch 87 - loss: 2.9315165443805364 - val_loss: 2.707514446147145
Epoch 90 - loss: 2.9046125173302806 - val_loss: 2.460648236513375
Epoch 96 - loss: 2.8046

In [290]:
y_pred = vregressor.predict(scaled_X_test)

In [292]:
y_pred

array([30.60780215, 35.10591942, 26.0054007 , 39.01270276, 15.89375584,
       17.99728717, 14.69126485, 36.4398758 , 13.47150772, 26.35626454,
       28.53587332, 24.07746348, 36.84360747, 13.88108429, 30.0586416 ,
       34.4046181 , 18.24526728, 34.13306707, 12.58403241, 23.16228996,
       19.36623327, 19.68528503, 26.54746929, 21.00745788, 35.11865951,
       21.76090962, 18.90002236, 26.32366799, 32.2704354 , 25.49646208,
       17.48770788, 17.8076253 , 12.43021684, 21.67811921, 12.12335115,
       37.42347901, 24.1966296 , 41.43846227, 22.1480613 , 15.14224081])

In [291]:
mean_absolute_error(y_test, y_pred)

1.8202937607925365

In [361]:
activation_function = np.vectorize(lambda x: 1 / (1 + math.exp(-x)))

w = [np.array([[[0, 3],[2, 0]],[[1, 3],[9, 7]],[[5, 8],[3, 3]]]),
    np.array([[[2],[6]],[[0],[4]],[[6],[9]]])]
#w1 = np.array([[[0, 3],[2, 0]],[[1, 3],[9, 7]],[[5, 8],[3, 3]]])
#w2 = np.array([[[2],[6]],[[0],[4]],[[6],[9]]])

np.random.seed(42)

B = np.array([  [8, 0],
                [9, 2],
                [6, 3]])

pred = []

for i in range(3):
    forward_pass = B.T
    forward_pass = activation_function(w[0][i].T @ forward_pass)

    forward_pass = w[1][i].T @ forward_pass

    pred.append(forward_pass.reshape(-1))



In [363]:
pred

[array([7.        , 7.96402758, 7.99505466]),
 array([4., 4., 4.]),
 array([15., 15., 15.])]

In [364]:


forward_pass = B.T
forward_pass = activation_function(w[0].transpose(0, 2, 1) @ forward_pass)
forward_pass = w[1].transpose(0, 2, 1) @ forward_pass
pred = forward_pass.reshape(3, 3)


In [365]:
pred.reshape(3, 3)

array([[ 7.        ,  7.96402758,  7.99505466],
       [ 4.        ,  4.        ,  4.        ],
       [15.        , 15.        , 15.        ]])

In [309]:
w2.shape

(3, 2, 1)

In [279]:
pred

[array([48, 58, 42, 72, 93, 72]),
 array([56, 73, 57, 16, 26, 24]),
 array([32, 54, 51,  0, 10, 15])]

In [282]:
np.random.randint(0, 10, (2, 2, (1, 1)), dtype = "object")

TypeError: Unsupported dtype dtype('O') for randint